In [43]:
import geopandas as gpd
import pandas as pd
import os
import duckdb
import numpy as np

from pathlib import Path
from zipfile import ZipFile
import tempfile
from shapely import wkt
import json
from geowrangler.datasets import geofabrik

import sys

sys.path.append("../../../")  # include parent directory
from src.vector_utils import *

# Add OSM Buildings to POIs

Join OSM buildings to the OSM POIs data for added data.

For this notebook, the extraction will need to be done per year rather than looping over all the years. `OSM Buildings` is a much larger file and looping over all the years will cause out-of-memory issues. The year has to be inputted manuallly.

In [44]:
DATA_DIR = Path("../../../data/")
DUCKDB_PATH = str(DATA_DIR / "duckdb.db")
SRC_DIR = Path("../../../src/")

In [45]:
YEAR = "2017"  # must change for every year extract
LOCAL_CRS = "EPSG:3857"
PROJ_CRS = "EPSG:4326"

OSM_DIR = Path("../../../data/02-raw/osm/")
OSM_UNZIP_DIR = OSM_DIR / YEAR
OSM_ZIP = OSM_DIR / f"philippines-{YEAR[2::]}0101-free.shp.zip"

ADMIN_BOUNDS = Path("../../../data/01-admin-bounds/target_admin_bounds.shp")

In [46]:
# filter for needed fclass/type
poi_types = Path(SRC_DIR / "osm_poi_categories.json")
with open(poi_types, "r") as f:
    data = json.load(f)
POI_TYPES_NEEDED = []
for category in data:
    POI_TYPES_NEEDED += data[category]

# filter for needed fclass/type
bldg_types = Path(SRC_DIR / "osm_bldgs_categories.json")
with open(bldg_types, "r") as f:
    data = json.load(f)
POIS_FROM_BLDGS = data["all_poi"]

to_compute = ["health", "crit_lifelines", "water", "sanitation"]
BLDG_TYPES_NEEDED = []
for category in data:
    if category in to_compute:
        BLDG_TYPES_NEEDED += data[category]

In [47]:
# to align OSM BLDGS with OSM POI types
RENAME_BLDG_DICT = {
    "civic": "public_building",
    "public": "public_building",
    "toilets": "toilet",
}

## Unzip OSM Buildings and POIs file

In [48]:
UNZIP = True  # change to True if need to unzip new file

# extract files
# loading the temp.zip and creating a zip object
if UNZIP:
    # make new directory to store unzipped files
    os.makedirs(OSM_UNZIP_DIR, exist_ok=True)

    with ZipFile(OSM_ZIP, "r") as zObject:

        file_list = zObject.namelist()

        for fname in file_list:
            if fname.startswith("gis_osm_buildings_a") or fname.startswith(
                "gis_osm_pois_free_1"
            ):
                # Extracting specific file in the zip
                # into a specific location.
                if not os.path.exists(OSM_UNZIP_DIR / fname):
                    zObject.extract(fname, path=OSM_UNZIP_DIR)
        zObject.close()

## Set-up duckdb

In [49]:
db = duckdb.connect(DUCKDB_PATH)
db.execute("INSTALL spatial")
db.execute(
    """
LOAD spatial;
"""
)

## Load AOI

In [50]:
aoi = gpd.read_file(ADMIN_BOUNDS)
aoi.head(2)

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053..."
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039..."


In [51]:
# create the aoi grids geodataframe into a duckdb table
with tempfile.NamedTemporaryFile(suffix=".geojson", delete=True) as tmpfile:
    aoi.to_file(tmpfile.name)

    query = f"""
    DROP TABLE IF EXISTS admin_bounds;
    CREATE TABLE IF NOT EXISTS admin_bounds as (
        SELECT 
            * EXCLUDE geom,
            geom as geometry
        FROM ST_Read('{tmpfile.name}')
    )
    ; 
    """
    db.execute(query).fetchdf()

query = f"""
DESCRIBE admin_bounds; 
"""
db.execute(query).fetchdf()

/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


,column_name,column_type,null,key,default,extra
0,ADM1_EN,VARCHAR,YES,None,None,NaN
1,ADM1_PCODE,VARCHAR,YES,None,None,NaN
2,ADM2_EN,VARCHAR,YES,None,None,NaN
3,ADM2_PCODE,VARCHAR,YES,None,None,NaN
4,ADM3_EN,VARCHAR,YES,None,None,NaN
5,ADM3_PCODE,VARCHAR,YES,None,None,NaN
6,ADM4_EN,VARCHAR,YES,None,None,NaN
7,ADM4_PCODE,VARCHAR,YES,None,None,NaN
8,geometry,GEOMETRY,YES,None,None,NaN


## Load tables

In [52]:
def load_pois_bldgs():
    # load pois
    pois_table_name = "osm_pois"
    query = f"""DROP TABLE if exists {pois_table_name};
                CREATE TABLE {pois_table_name} as FROM ST_READ("{OSM_UNZIP_DIR}/gis_osm_pois_free_1.shp")"""
    db.execute(query)

    # load buildings
    bldgs_table_name = "osm_bldgs"
    query = f"""DROP TABLE if exists {bldgs_table_name};
                CREATE TABLE {bldgs_table_name} as FROM ST_READ("{OSM_UNZIP_DIR}/gis_osm_buildings_a_free_1.shp")"""
    db.execute(query)

    return pois_table_name, bldgs_table_name

In [53]:
pois_table_name, bldgs_table_name = load_pois_bldgs()

## Clipping

In [54]:
def clip_osm_files(pois_table, bldgs_table):
    # create geometry for cropping
    aoi_wkt = aoi.unary_union
    # crop POIs file
    query = f"""
    DROP TABLE IF EXISTS clipped_{pois_table};
    CREATE TABLE IF NOT EXISTS clipped_{pois_table} as (
        SELECT
            osm_id,
            fclass,
            name,
            ST_AsText(geom) AS geom
        FROM {pois_table}
        WHERE 
            ST_Intersects(geom, ST_GeomFromText('{aoi_wkt}'))
    );
    """
    db.execute(query)
    clipped_osm_pois_df = db.execute(
        f"""SELECT * FROM clipped_{pois_table}"""
    ).fetchdf()

    # crop bldgs file
    query = f"""
    DROP TABLE IF EXISTS clipped_{bldgs_table};
    CREATE TABLE IF NOT EXISTS clipped_{bldgs_table} as (
        SELECT
            osm_id,
            fclass,
            name,
            type,
            ST_AsText(geom) AS geom
        FROM {bldgs_table}
        WHERE 
            ST_Intersects(geom, ST_GeomFromText('{aoi_wkt}'))
    );
    """
    db.execute(query)
    clipped_osm_bldgs_df = db.execute(
        f"""SELECT * FROM clipped_{bldgs_table}"""
    ).fetchdf()

    return clipped_osm_pois_df, clipped_osm_bldgs_df

In [55]:
%%time
clipped_pois, clipped_bldgs = clip_osm_files(pois_table_name, bldgs_table_name)

CPU times: user 4min 48s, sys: 600 ms, total: 4min 49s
Wall time: 30.2 s


## Prepare OSM POIs 

Select only the points we would need

In [56]:
def preprocess_dfs(clipped_pois_df, clipped_bldgs_df):
    # pois
    clipped_pois_df["geom"] = clipped_pois_df["geom"].apply(wkt.loads)
    clipped_osm_pois_gdf = gpd.GeoDataFrame(
        clipped_pois_df, geometry="geom", crs=PROJ_CRS
    )
    # buildings
    clipped_bldgs_df["geom"] = clipped_bldgs_df["geom"].apply(wkt.loads)
    clipped_osm_bldgs_gdf = gpd.GeoDataFrame(
        clipped_bldgs_df, geometry="geom", crs=PROJ_CRS
    )
    clipped_osm_bldgs_gdf = clipped_osm_bldgs_gdf.dropna(subset=["type"])
    clipped_osm_bldgs_gdf = clipped_osm_bldgs_gdf[
        clipped_osm_bldgs_gdf["type"].isin(POIS_FROM_BLDGS)
    ]

    return clipped_osm_pois_gdf, clipped_osm_bldgs_gdf


def deduplicate_points(clipped_pois_df, clipped_bldgs_df):
    pois_gdf, bldgs_gdf = preprocess_dfs(clipped_pois_df, clipped_bldgs_df)
    # pois
    pois_gdf = pois_gdf.to_crs(LOCAL_CRS)
    pois_gdf["geometry_buffer"] = pois_gdf.buffer(10)  # 10m buffer
    pois_gdf = pois_gdf.set_geometry("geometry_buffer")

    # buildings
    bldgs_gdf = bldgs_gdf.to_crs(LOCAL_CRS)
    bldgs_gdf["geom_centroid"] = bldgs_gdf.centroid
    bldgs_gdf = bldgs_gdf.set_geometry("geom_centroid")
    bldgs_gdf["geom_centroid_buffered"] = bldgs_gdf.buffer(10)  # 10m buffer
    bldgs_gdf = bldgs_gdf.set_geometry("geom_centroid_buffered")

    # get duplicates
    duplicates = gpd.sjoin(pois_gdf, bldgs_gdf, how="inner", predicate="intersects")
    duplicates_to_remove = duplicates["osm_id_right"].tolist()

    # clean bldgs
    bldgs_gdf = bldgs_gdf[
        ~bldgs_gdf["osm_id"].isin(duplicates_to_remove)
    ]  # remove what's in POIs already
    bldgs_gdf = bldgs_gdf.set_geometry("geom_centroid")
    bldgs_gdf = bldgs_gdf.drop(columns=["geom", "geom_centroid_buffered", "fclass"])
    bldgs_gdf = bldgs_gdf.rename(columns={"type": "fclass", "geom_centroid": "geom"})
    bldgs_gdf["fclass"] = bldgs_gdf["fclass"].replace(RENAME_BLDG_DICT)
    bldgs_gdf = bldgs_gdf.set_geometry("geom")
    bldgs_gdf = bldgs_gdf.to_crs(PROJ_CRS)
    print(f"Number of buildings to be added: {bldgs_gdf.shape[0]}")

    # clean pois
    pois_gdf = pois_gdf.set_geometry("geom")
    pois_gdf = pois_gdf.drop(columns=["geometry_buffer"])
    pois_gdf = pois_gdf.to_crs(PROJ_CRS)
    print(f"Number of POIs currently: {pois_gdf.shape[0]}")

    updated_gdf = gpd.GeoDataFrame(pd.concat([pois_gdf, bldgs_gdf], ignore_index=True))
    return updated_gdf

## Deduplication

In [57]:
%%time

osm_updated_points = deduplicate_points(clipped_pois, clipped_bldgs)
osm_updated_points.head(3)

Number of buildings to be added: 1542
Number of POIs currently: 8152
CPU times: user 4.67 s, sys: 422 µs, total: 4.67 s
Wall time: 4.66 s


,osm_id,fclass,name,geom
0,25679317,restaurant,La Roca Suites & Restaurant,POINT (123.74645 13.15615)
1,25679322,park,Renaissance Garden,POINT (123.73565 13.15147)
2,25679364,post_office,Legaspi City Post Office,POINT (123.74891 13.15538)


In [58]:
osm_updated_points = osm_updated_points.set_crs(PROJ_CRS)

In [59]:
osm_updated_points.shape

(9694, 4)

In [60]:
%%time
updated_feats_gdf = add_point_features(
    aoi, osm_updated_points, types_col="fclass", poi_types=POI_TYPES_NEEDED
)
updated_feats_gdf.head(3)

CPU times: user 10.2 s, sys: 40.3 ms, total: 10.3 s
Wall time: 10.3 s


,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,poi_count,...,toilet_count,toilet_nearest,recycling_count,recycling_nearest,waste_basket_count,waste_basket_nearest,wastewater_plant_count,wastewater_plant_nearest,waste_transfer_station_count,waste_transfer_station_nearest
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",0.0,...,0.0,10000.0,0.0,2486.499185,0.0,10000.0,0.0,10000.0,0.0,10000.0
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",52.0,...,0.0,10000.0,0.0,79.395847,0.0,10000.0,0.0,10000.0,0.0,10000.0
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Pantal,PH015518022,"POLYGON ((120.34737 16.06009, 120.34761 16.060...",84.0,...,0.0,10000.0,0.0,1752.273027,0.0,10000.0,0.0,10000.0,0.0,10000.0


In [61]:
updated_feats_gdf.columns

Index(['ADM1_EN', 'ADM1_PCODE', 'ADM2_EN', 'ADM2_PCODE', 'ADM3_EN',
       'ADM3_PCODE', 'ADM4_EN', 'ADM4_PCODE', 'geometry', 'poi_count',
       ...
       'toilet_count', 'toilet_nearest', 'recycling_count',
       'recycling_nearest', 'waste_basket_count', 'waste_basket_nearest',
       'wastewater_plant_count', 'wastewater_plant_nearest',
       'waste_transfer_station_count', 'waste_transfer_station_nearest'],
      dtype='object', length=126)

## Export and Close connection

In [62]:
updated_feats_gdf = updated_feats_gdf.drop(
    columns=[
        "ADM1_EN",
        "ADM1_PCODE",
        "ADM2_EN",
        "ADM2_PCODE",
        "ADM3_EN",
        "ADM3_PCODE",
        "ADM4_EN",
        "geometry",
    ]
)
updated_feats_gdf.head(2)

,ADM4_PCODE,poi_count,clinic_count,clinic_nearest,dentist_count,dentist_nearest,doctors_count,doctors_nearest,hospital_count,hospital_nearest,...,toilet_count,toilet_nearest,recycling_count,recycling_nearest,waste_basket_count,waste_basket_nearest,wastewater_plant_count,wastewater_plant_nearest,waste_transfer_station_count,waste_transfer_station_nearest
0,PH015518016,0.0,0.0,10000.0,0.0,1769.374267,0.0,10000.0,0.0,1287.610535,...,0.0,10000.0,0.0,2486.499185,0.0,10000.0,0.0,10000.0,0.0,10000.0
1,PH015518031,52.0,0.0,10000.0,1.0,0.000000,0.0,10000.0,2.0,0.000000,...,0.0,10000.0,0.0,79.395847,0.0,10000.0,0.0,10000.0,0.0,10000.0


In [63]:
updated_feats_gdf.to_csv(OSM_DIR / f"osm-poi-updated-feat-{YEAR}.csv")